In [2]:
# from pycaret.time_series import *
# from supervised.automl import AutoML

import pandas as pd
import numpy as np
import category_encoders as ce
import copy
import polars as pl

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
import plotly.express as px
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
# from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
# from sklearn.base import ClassifierMixin

# CatBoost
from catboost import CatBoostRegressor

# PyTorch
# import torch
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
# from torch.nn import Parameter
# from torch import Tensor
# from torch.utils.data import DataLoader
# from torch.optim.lr_scheduler import ReduceLROnPlateau

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

import warnings

warnings.filterwarnings("ignore")

import os
import gc
import datetime
import numpy as np
import pandas as pd
import polars as pl
import plotly.express as px
import joblib
from sklearn.ensemble import VotingRegressor
import lightgbm as lgb
import holidays

In [3]:
# root = "/kaggle/input/predict-energy-behavior-of-prosumers"
root = '../data' # my root

df_train = pd.read_csv(os.path.join(root, "train.csv"))
df_client = pd.read_csv(os.path.join(root, "client.csv"))
df_gas_prices = pd.read_csv(os.path.join(root, "gas_prices.csv"))
df_electricity_prices = pd.read_csv(os.path.join(root, "electricity_prices.csv"))
df_forecast_weather = pd.read_csv(os.path.join(root, "forecast_weather.csv"))
df_historical_weather = pd.read_csv(os.path.join(root, "historical_weather.csv"))
df_weather_station_to_county_mapping = pd.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"))

In [4]:
segments_name = ['county','is_business','is_consumption','product_type']
seg_dict = {}
for seg in segments_name:
    seg_dict[seg] = df_train[seg].unique()

In [5]:
seg_dict

{'county': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       dtype=int64),
 'is_business': array([0, 1], dtype=int64),
 'is_consumption': array([0, 1], dtype=int64),
 'product_type': array([1, 2, 3, 0], dtype=int64)}

In [6]:
import itertools
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [6]:
# df_train_plot_sorted = pd.read_csv('../data/made_data/df_train_plot_sorted.csv')
# df_train_plot_sorted.head()

,Unnamed: 0,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
0,0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,0_0_0
1,165458,0,0,2,0.154,0,2021-10-26 16:00:00,55,165458,1,0_0_0
2,165456,0,0,1,2.742,0,2021-10-26 16:00:00,55,165456,0,0_0_0
3,736900,0,0,3,5225.561,0,2022-04-27 12:00:00,238,736900,2,0_0_0
4,1191032,0,0,3,1952.719,0,2022-09-16 14:00:00,380,1191032,2,0_0_0


In [8]:
df_train_plot = df_train.copy()
df_train_plot['seg_index'] = ''
def make_seg(row):
    row['seg_index'] = f"{row['is_business']}_{row['product_type']}_{row['is_consumption']}"
    return row
df_train_plot = df_train_plot.apply(make_seg,axis= 1)

In [18]:
# df_train_plot.to_csv('../data/made_data/df_train_plot_sorted.csv')

In [9]:
df_train_plot_sorted=df_train_plot.sort_values('seg_index')
df_train_plot_sorted.to_csv('../data/made_data/df_train_plot_sorted_county.csv')
df_train_plot_sorted.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,0_1_0
1605090,14,0,1,0.000,0,2023-01-22 00:00:00,508,1605090,53,0_1_0
992386,13,0,1,0.000,0,2022-07-16 02:00:00,318,992386,50,0_1_0
374628,0,0,1,0.000,0,2022-01-03 18:00:00,124,374628,0,0_1_0
1970446,9,0,1,0.055,0,2023-05-16 22:00:00,622,1970446,34,0_1_0


In [ ]:
# i = 0 
# n_cols = df_train_plot_sorted.seg_index.nunique()
# sub_cols = 2
# sub_rows = (n_cols+1) // 2
# width = n_cols  # 2의 배수 곱해서 사이즈 조절
# height = sub_rows * 4  # 2의 배수 곱해서 사이즈 조절

# plt.figure(figsize=(width, height)) # figure의 width height는 맘대로

# uniq_l = df_train_plot_sorted['seg_index'].unique()
# for seg_index in tqdm(uniq_l):
#     subset = df_train_plot_sorted[df_train_plot_sorted['seg_index']==seg_index].copy()
#     i += 1
#     plt.subplot(sub_rows, sub_cols, i)
#     sns.lineplot(data=subset,x='datetime',y='target',hue='product_type',errorbar=('ci',95))
#     plt.xticks(rotation=45)
# plt.tight_layout()   
# plt.show()
# plt.savefig('../plots/target_lineplot_by_segments.png',dpi=300)

In [7]:
df_train_plot_sorted['product_type'].unique()

array([1, 2, 3, 0], dtype=int64)

In [10]:
temp = df_train_plot_sorted.copy()
# temp['target'] = np.log1p(temp['target'])
temp.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,0_1_0
1605090,14,0,1,0.000,0,2023-01-22 00:00:00,508,1605090,53,0_1_0
992386,13,0,1,0.000,0,2022-07-16 02:00:00,318,992386,50,0_1_0
374628,0,0,1,0.000,0,2022-01-03 18:00:00,124,374628,0,0_1_0
1970446,9,0,1,0.055,0,2023-05-16 22:00:00,622,1970446,34,0_1_0


In [11]:
i = 0 
n_cols = temp.seg_index.nunique()

uniq_l = temp['seg_index'].unique()
for seg_index in tqdm(uniq_l):
    subset = temp[temp['seg_index']==seg_index].copy()
    i += 1
    sns.lineplot(data=subset,x='datetime',y='target',hue='county',
                 errorbar=None)
    plt.savefig(f'../plots/target_by_seg_lineplot/target_lineplot_by_segments_{seg_index}_b_p_c.png')
    plt.clf()
    subset = pd.DataFrame()
# plt.tight_layout()       
# plt.show()
# plt.savefig('../plots/target_lineplot_by_segments.png',dpi=300)

100%|████████████████████████████████████████████████| 14/14 [29:15<00:00, 125.41s/it]


<Figure size 640x480 with 0 Axes>

In [8]:
uniq_l = df_train_plot_sorted['seg_index'].unique()
x = 1
for seg_index in tqdm(uniq_l):
    subset = df_train_plot_sorted[df_train_plot_sorted['seg_index']==seg_index].copy()
    display(subset.head(20))
    x+=1
    if x == 6: break;

  0%|                                                                      | 0/60 [00:00<?, ?it/s]

,Unnamed: 0,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
0,0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,0_0_0
1,165458,0,0,2,0.154,0,2021-10-26 16:00:00,55,165458,1,0_0_0
2,165456,0,0,1,2.742,0,2021-10-26 16:00:00,55,165456,0,0_0_0
3,736900,0,0,3,5225.561,0,2022-04-27 12:00:00,238,736900,2,0_0_0
4,1191032,0,0,3,1952.719,0,2022-09-16 14:00:00,380,1191032,2,0_0_0
5,1191030,0,0,2,25.499,0,2022-09-16 14:00:00,380,1191030,1,0_0_0
6,1191028,0,0,1,626.799,0,2022-09-16 14:00:00,380,1191028,0,0_0_0
7,1834238,0,0,1,2.342,0,2023-04-03 23:00:00,579,1834238,0,0_0_0
8,165460,0,0,3,19.848,0,2021-10-26 16:00:00,55,165460,2,0_0_0
9,1834240,0,0,2,0.000,0,2023-04-03 23:00:00,579,1834240,1,0_0_0


,Unnamed: 0,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
45936,1780909,0,0,1,836.908,1,2023-03-18 03:00:00,563,1780909,0,0_0_1
45937,28915,0,0,1,134.794,1,2021-09-10 21:00:00,9,28915,0,0_0_1
45938,566161,0,0,1,130.632,1,2022-03-05 12:00:00,185,566161,0,0_0_1
45939,1237263,0,0,3,1185.088,1,2022-09-30 23:00:00,394,1237263,2,0_0_1
45940,677059,0,0,2,31.611,1,2022-04-09 04:00:00,220,677059,1,0_0_1
45941,334675,0,0,1,443.301,1,2021-12-21 15:00:00,111,334675,0,0_0_1
45942,1429795,0,0,1,859.508,1,2022-11-28 13:00:00,453,1429795,0,0_0_1
45943,1172809,0,0,3,1073.917,1,2022-09-10 22:00:00,374,1172809,2,0_0_1
45944,1240659,0,0,2,11.206,1,2022-10-02 00:00:00,396,1240659,1,0_0_1
45945,81377,0,0,2,36.022,1,2021-09-28 19:00:00,27,81377,1,0_0_1


  3%|██                                                            | 2/60 [00:00<00:05, 10.07it/s]

,Unnamed: 0,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
91872,1191308,0,1,3,322.827,0,2022-09-16 16:00:00,380,1191308,5,0_1_0
91873,165846,0,1,3,0.652,0,2021-10-26 19:00:00,55,165846,5,0_1_0
91874,782634,0,1,2,10.338,0,2022-05-11 16:00:00,252,782634,61,0_1_0
91875,1452502,0,1,1,0.037,0,2022-12-05 13:00:00,460,1452502,4,0_1_0
91876,736500,0,1,3,2469.710,0,2022-04-27 09:00:00,238,736500,5,0_1_0
91877,1871394,0,1,2,0.051,0,2023-04-15 20:00:00,591,1871394,61,0_1_0
91878,1871390,0,1,0,0.200,0,2023-04-15 20:00:00,591,1871390,3,0_1_0
91879,1873866,0,1,3,5363.666,0,2023-04-16 15:00:00,592,1873866,5,0_1_0
91880,1871392,0,1,1,0.700,0,2023-04-15 20:00:00,591,1871392,4,0_1_0
91881,1652270,0,1,3,77.270,0,2023-02-05 19:00:00,522,1652270,5,0_1_0


,Unnamed: 0,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
152400,393747,0,1,1,907.256,1,2022-01-09 21:00:00,130,393747,4,0_1_1
152401,1048651,0,1,0,245.780,1,2022-08-02 21:00:00,335,1048651,3,0_1_1
152402,1689837,0,1,2,148.927,1,2023-02-17 11:00:00,534,1689837,61,0_1_1
152403,576337,0,1,3,10289.586,1,2022-03-08 17:00:00,188,576337,5,0_1_1
152404,136737,0,1,1,672.412,1,2021-10-17 04:00:00,46,136737,4,0_1_1
152405,857539,0,1,1,699.447,1,2022-06-03 23:00:00,275,857539,4,0_1_1
152406,576205,0,1,3,11322.519,1,2022-03-08 16:00:00,188,576205,5,0_1_1
152407,220531,0,1,3,5644.340,1,2021-11-13 21:00:00,73,220531,5,0_1_1
152408,1149763,0,1,0,237.060,1,2022-09-03 18:00:00,367,1149763,3,0_1_1
152409,72721,0,1,1,640.746,1,2021-09-25 20:00:00,24,72721,4,0_1_1


  7%|████▏                                                         | 4/60 [00:00<00:05,  9.84it/s]

,Unnamed: 0,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,seg_index
212928,1557502,10,0,3,0.057,0,2023-01-07 04:00:00,493,1557502,39,10_0_0
212929,645400,10,0,1,39.415,0,2022-03-30 09:00:00,210,645400,38,10_0_0
212930,1297046,10,0,3,0.747,0,2022-10-19 06:00:00,413,1297046,39,10_0_0
212931,1647652,10,0,1,0.522,0,2023-02-04 08:00:00,521,1647652,38,10_0_0
212932,1647654,10,0,3,0.508,0,2023-02-04 08:00:00,521,1647654,39,10_0_0
212933,240508,10,0,3,232.803,0,2021-11-20 11:00:00,80,240508,39,10_0_0
212934,1623846,10,0,1,0.000,0,2023-01-27 22:00:00,513,1623846,38,10_0_0
212935,409292,10,0,3,0.000,0,2022-01-14 20:00:00,135,409292,39,10_0_0
212936,1339074,10,0,1,0.046,0,2022-11-01 03:00:00,426,1339074,38,10_0_0
212937,146258,10,0,1,0.000,0,2021-10-20 07:00:00,49,146258,38,10_0_0


  7%|████▏                                                         | 4/60 [00:00<00:07,  7.86it/s]


In [14]:
df_train[(df_train['county']=='12')&
        (df_train['is_business']=='1')&
        (df_train['product_type']=='0')]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
